In [1]:
import Visualization as viz
from ipywidgets import widgets
from IPython.display import display, clear_output
from bokeh.plotting import figure, show, output_notebook
import pandas as pd

output_notebook()

Loading BokehJS ...

In [2]:
from pymongo import MongoClient

client = MongoClient()
db = client.VentDB
breath_db = db.breath_collection
train_db = db.train_collection

train_db.find().count()

10024

In [3]:
breath_db.find().count()

1964338

In [4]:
def on_button_click(b):
    global breaths

    clear_output()
    if results.alive:
        if breaths != 1:
            viz.update_database(breaths['_id'], w.selected_labels)

        breaths = results.next()
        df, breath_start, breath_end = viz.breath_viz(breaths['_id'])
        p=viz.make_plot(df, breath_start, breath_end)
        show(p)
    else:
        if breaths != 1:
            viz.update_database(breaths['_id'], w.selected_labels)
        clear_output()
        print('No more')

In [5]:
train_db.find({'validation':{'$exists':1}}).count()

1873

In [6]:
import pandas as pd
val_df = pd.io.json.json_normalize(list(train_db.find({'validation':{'$exists':1}}, {'validation':1})))

In [7]:
val_df.replace({0:False, 1:True}, inplace=True)
val_df = val_df[['validation.ds', 'validation.fa', 'validation.garb',
       'validation.ie', 'validation.norm', 'validation.pds', 'validation.pfl',
       'validation.pvt']].astype(bool)

In [8]:
val_df['validation.ds'].value_counts()

False    1721
True      152
Name: validation.ds, dtype: int64

In [9]:
val_df['validation.pds'].value_counts()

False    1849
True       24
Name: validation.pds, dtype: int64

In [10]:
val_df['validation.pfl'].value_counts()

False    1403
True      470
Name: validation.pfl, dtype: int64

In [11]:
val_df['validation.pvt'].value_counts()

False    1775
True       98
Name: validation.pvt, dtype: int64

In [12]:
val_df['validation.ie'].value_counts()

False    1549
True      324
Name: validation.ie, dtype: int64

In [13]:
val_df['validation.norm'].value_counts()

False    989
True     884
Name: validation.norm, dtype: int64

In [14]:
breaths = 1
results = viz.get_breaths(75)
w = widgets.SelectMultiple(description='Types', options=['Normal', 'DoubleStacked', 'PostDoubleStacked', 
                                                         'PressureFlowLimited', 'FlowAbnormal', 
                                                         'PrematureVentTermination', 'IneffectiveTrigger', 'Garbage'])
w.height = 150
button = widgets.Button(description='Next')
container = widgets.HBox(children=[w, button])

button.on_click(on_button_click)
display(container)

breath error: 119/8350/2015-03-08 07:54:09/40374982
[{'file': '/media/veracrypt1/Research_Data/VentDysSedation/RawData/P119/hm_Waveform 15-03-07-20-41.txt', 'breath_raw': {'sm_flow': [9.900000000000007, 19.357142857142872, 28.071428571428594, 35.71428571428575, 47.10000000000004, 53.435714285714326, 58.43571428571434, 60.385714285714336, 61.535714285714334, 57.464285714285765, 51.021428571428615, 42.46428571428575, 19.271428571428586, -7.285714285714291, -31.071428571428598, -48.45000000000003, -55.76428571428576, -54.464285714285765, -59.06428571428577, -61.7428571428572, -62.81428571428577, -62.18571428571434, -58.7571428571429, -57.19285714285719, -55.83571428571433, -55.77142857142863, -55.214285714285765, -54.25000000000005, -52.79285714285719, -53.200000000000045, -52.88571428571433, -52.185714285714326, -49.97857142857147, -45.521428571428615, -39.578571428571465, -32.17857142857145, -22.907142857142876, -12.835714285714298, -3.228571428571432, 4.50714285714286], 'time': [403749

# Use PDS breaths to ID DS Breaths

In [32]:
pds = list(train_db.find({'validation':{'$exists':1}, 'validation.pds':1}, {'patient_id':1, 'date_time':1, 'breath_num':1}))
for breath in pds:
    ids = breath_db.find_one({'patient_id': breath['patient_id'], 
                              'date_time': breath['date_time'], 'breath_num':breath['breath_num']}, {'file':1})
    data = breath_db.find_one({'patient_id': breath['patient_id'], 'file': ids['file'], 'breath_num':breath['breath_num']-1})
    data['validation'] = {'norm': 0,
                                'ds': 1,
                                'pds': 0,
                                'pfl': 0,
                                'fa': 0,
                                'pvt':0,
                                'ie': 0,
                                'garb': 0}

    try:
        train_db.insert_one(data)
    except:
        train_db.update_one({'patient_id': breath['patient_id'], 
                             'file':ids['file'], 'breath_num':breath['breath_num']-1}, {'$set':data})

In [ ]:
ids = list(train_db.find({'validation':{'$exists':1}}, {'_id':1, 'validation':1}).limit(20))

In [ ]:
for _id in ids:
    print(_id['validation'])
    df, breath_start, breath_end = viz.breath_viz(_id['_id'])
    p=viz.make_plot(df, breath_start, breath_end)
    show(p)

## Pull in additional data from database

In [33]:
import datetime

pds = list(train_db.find({'validation':{'$exists':1}}, {'patient_id':1, 'date_time':1, 'breath_num':1, 'validation':1}))
for breath in pds:

    if isinstance(breath['date_time'], float):
        date_time = datetime.datetime.fromtimestamp(breath['date_time'])
        data = breath_db.find({'patient_id': breath['patient_id'], 
                              'date_time': date_time, 'breath_num':breath['breath_num']})
    else: 
        data = breath_db.find({'patient_id': breath['patient_id'], 
                              'date_time': breath['date_time'], 'breath_num':breath['breath_num']})
    
    if data.count() == 1:
        data = list(data)[0]
        data['validation'] = breath['validation']
        data.pop('_id')
        train_db.update_one({'_id':breath['_id']}, {'$set':data})
    else:
        date = re.search('\d\d\d\d-\d\d-\d\d \d\d:\d\d:\d\d', breath['_id']).group()
        date_time = datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        data = breath_db.find({'patient_id': breath['patient_id'], 
                              'date_time': date_time, 'breath_num':breath['breath_num']}) 
        if data.count() == 1:
            data = list(data)[0]
            data['validation'] = breath['validation']
            data.pop('_id')
            train_db.update_one({'_id':breath['_id']}, {'$set':data})
        else:
            print(breath)
            print('Missing data')